In [62]:
import gymnasium as gym
import numpy as np
import torch
from torch import nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.distributions import Categorical
from tqdm import tqdm
from torch.optim import Adam

In [63]:
network = nn.Sequential(*[nn.Linear(2, 30), 
                          nn.SELU(),
                          nn.Linear(30, 3),
                          nn.Softmax(-1),
                         ]); ### This dude is the policy function
state_value = nn.Sequential(*[nn.Linear(2, 30), 
                          nn.SELU(),
                          nn.Linear(30, 30),
                          nn.SELU(),
                          nn.Linear(30, 1)
                             ]) ### This dude is the state value

In [64]:
@torch.no_grad
def choose_action(state:np.ndarray, network:nn.Module = network)->int:
    probs = network(torch.tensor(state))
    probs = Categorical(probs).sample()
    return probs.item()

In [ ]:
env = gym.make("MountainCar-v0", max_episode_steps = 2000)

γ = 0.98

opt1 = Adam(network.parameters(), 0.00001)
opt2 = Adam(state_value.parameters(), 0.00001)

num_iters = tqdm(range(50))

for i in num_iters:
    
    last_step, info = env.reset()
    ### This dude is kind a replay buffer!!!
    for c in range(2000):
        action = choose_action(last_step, network)  # agent policy that uses the observation and info
        current_state, reward, terminated, truncated, info = env.step(action)
    
        with torch.no_grad():
            δ = reward + γ*(1-1*terminated)*state_value(torch.tensor(current_state)) - state_value(torch.tensor(last_state))
        
        opt1.zero_grad()
        opt2.zero_grad()
        
        z = -δ*state_value(torch.tensor(last_state))
        z.backward()
        z_ = -((δ*(network(torch.tensor(last_state))[action])).log()).squeeze()
        z_.backward()

        opt1.step()
        opt2.step()
        
        last_state = current_state
    
env.close()
    #update_params(network, state_value,states, actions, rewards)

 28%|██████████████████████▉                                                           | 14/50 [00:47<02:05,  3.49s/it]

In [1]:
env = gym.make("MountainCar-v0", max_episode_steps = 300, render_mode = "human")
state, info = env.reset()

for c in range(300):
    network.eval()
    action = choose_action(state, network)  # agent policy that uses the observation and info
    state, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        state, info = env.reset()
        print(c)
        break
    
env.close()

NameError: name 'gym' is not defined